<a href="https://colab.research.google.com/github/Ais2004/Ais2004/blob/main/Rail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Load dataset
df = pd.read_csv("train_speed_data.csv")

# Features & Target Variable
X = df.drop(columns=["Recommended_Speed"])
y = df["Recommended_Speed"]

# Identify categorical and numerical features
categorical_features = ["Track_Condition", "Weather"]
numerical_features = ["Current_Speed", "Obstacle_Detected", "Station_Proximity"]

# Preprocessing: One-Hot Encoding for categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        ("num", "passthrough", numerical_features),
    ]
)

# Model Pipeline
model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", LinearRegression())
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Save the trained model
with open("train_speed_model.pkl", "wb") as file:
    pickle.dump(model, file)

print("✅ Model training complete! Saved as train_speed_model.pkl")


✅ Model training complete! Saved as train_speed_model.pkl


# New section

In [87]:
# Load the trained model
with open("train_speed_model.pkl", "rb") as file:
    model = pickle.load(file)

print(f"Loaded Model Type: {type(model)}")  # Should be <class 'sklearn.pipeline.Pipeline'>


Loaded Model Type: <class 'sklearn.pipeline.Pipeline'>


In [88]:
%%writefile predict_speed.py
import pickle
import pandas as pd

# Load the trained model
with open("train_speed_model.pkl", "rb") as file:
    model = pickle.load(file)

# Function to predict train speed
def predict_speed(current_speed, track_condition, weather, obstacle_detected, station_proximity):
    # Create a dataframe with input values
    input_data = pd.DataFrame([{
        "Current_Speed": current_speed,
        "Track_Condition": track_condition,
        "Weather": weather,
        "Obstacle_Detected": obstacle_detected,
        "Station_Proximity": station_proximity
    }])

    # Predict the speed
    predicted_speed = model.predict(input_data)[0]
    return predicted_speed

# Example Usage
if __name__ == "__main__":
    speed = predict_speed(80, "Moderate", "Clear", 0, 2.5)
    print(f"Predicted Recommended Speed: {speed:.2f} km/h")


Overwriting predict_speed.py


In [89]:
!python predict_speed.py


Predicted Recommended Speed: 78.95 km/h


In [90]:
!pip install flask


In [71]:
!pip install pyngrok


In [72]:
!ngrok authtoken 2si6kwyBplAMpVHynRwVF7M7yeL_7sF9ZTR9kA3f5rVLU7ncz


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [66]:
!pip install flask flask-ngrok


In [79]:
!pip install pyngrok
from pyngrok import ngrok

# Replace YOUR_AUTH_TOKEN with the token from https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token("2si6kwyBplAMpVHynRwVF7M7yeL_7sF9ZTR9kA3f5rVLU7ncz")


In [73]:
%%writefile app.py
import pickle
import pandas as pd
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

# Load the trained model
with open("train_speed_model.pkl", "rb") as file:
    model = pickle.load(file)

# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)  # This makes Flask run in Google Colab

# Prediction Route
@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get JSON data from request
        data = request.get_json()

        # Convert input to DataFrame
        input_data = pd.DataFrame([{
            "Current_Speed": data["current_speed"],
            "Track_Condition": data["track_condition"],
            "Weather": data["weather"],
            "Obstacle_Detected": data["obstacle_detected"],
            "Station_Proximity": data["station_proximity"]
        }])

        # Predict speed
        predicted_speed = model.predict(input_data)[0]

        # Return JSON response
        return jsonify({"recommended_speed": round(predicted_speed, 2)})

    except Exception as e:
        return jsonify({"error": str(e)})

# Run the app
if __name__ == "__main__":
    app.run()


Overwriting app.py


In [80]:
!python app.py &


 * Serving Flask app 'app'
 * Debug mode: off
Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [91]:
!lsof -i :5000


COMMAND   PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
python3 21946 root    3u  IPv4 616273      0t0  TCP localhost:5000 (LISTEN)


In [92]:
!kill -9 PID


/bin/bash: line 1: kill: PID: arguments must be process or job IDs


In [93]:
!python app.py --port=8081 &


 * Serving Flask app 'app'
 * Debug mode: off
Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [94]:
from pyngrok import ngrok
public_url = ngrok.connect(8081)
print("Your public URL:", public_url)


Your public URL: NgrokTunnel: "https://0657-34-80-159-217.ngrok-free.app" -> "http://localhost:8081"


In [95]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [96]:
%cd /content/drive/My Drive


/content/drive/My Drive


In [97]:
import os
os.makedirs("/content/drive/My Drive/FlaskProject", exist_ok=True)


In [98]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def home():
    return "Hello, Flask is running on Google Colab!"

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/serving.py", line 759, in __init__
    self.server_bind()
  File "/usr/lib/python3.11/http/server.py", line 136, in server_bind
    socketserver.TCPServer.server_bind(self)
  File "/usr/lib/python3.11/socketserver.py", line 472, in server_bind
    self.socket.bind(self.server_address)
OSError: [Errno 98] Address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-98-d86bc478e49c>", line 10, in <cell line: 0>
    app.run()
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 662, in run
    run_simple(t.cast(str, host), port, self, **options)
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/serving.py", line 1093, in run_simple

TypeError: object of type 'NoneType' has no len()